In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/san-guo-txt/san_guo.txt


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
from torch.nn.attention import SDPBackend, sdpa_kernel

In [3]:
def get_batch(split):
    # 选择训练或验证数据集
    data = train_data if split == 'train' else val_data

    # 动态从数据集中选择位置索引
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])

    return x.to(device),y.to(device)

In [4]:
class Block(nn.Module):
    def __init__(self, n_embd, n_head, dropout):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, n_embd, head_size, dropout)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        # 确保残差连接数据类型一致
        x = x + self.sa(self.ln1(x)).to(x.dtype)   # 残差连接
        x = x + self.ffwd(self.ln2(x)).to(x.dtype) # 残差连接
        return x

In [5]:
class FeedFoward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, n_embd*4),
            nn.ReLU(),
            nn.Linear(4*n_embd, n_embd),
        )

    def forward(self, x):
        return self.net(x.float())  # 确保输入为float32

In [6]:
class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, n_embd, head_embd, dropout):
        super().__init__()
        self.heads = nn.ModuleList([Head(n_embd, head_embd, dropout) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # 确保输入和输出数据类型一致
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.proj(out.float())  # 确保转换为float32
        return out

In [ ]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, n_embd, head_embd, dropout):
        super().__init__()
        self.key = nn.Linear(n_embd, head_embd, bias=False)
        self.query = nn.Linear(n_embd, head_embd, bias=False)
        self.value = nn.Linear(n_embd, head_embd, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)
        self.head_embd = head_embd

    def forward(self, input_x):
        B, T, C = input_x.shape
        k = self.key(input_x)   # (B, T, head_size)
        q = self.query(input_x) # (B, T, head_size)
        v = self.value(input_x) # (B, T, head_size)

        # 确保所有张量在相同设备上
        device = q.device

        # 尝试使用Flash Attention（需要4维输入）
        if torch.cuda.is_available():
            try:
                # 转换为4维 (B, num_heads, T, head_size)
                q_4d = q.view(B, T, 1, self.head_embd).transpose(1, 2)  # (B, 1, T, head_size)
                k_4d = k.view(B, T, 1, self.head_embd).transpose(1, 2)
                v_4d = v.view(B, T, 1, self.head_embd).transpose(1, 2)

                # 确保数据类型一致（全部转为float16）
                q_4d, k_4d, v_4d = q_4d.half(), k_4d.half(), v_4d.half()

                with torch.backends.cuda.sdp_kernel(enable_flash=True):
                    attn_output = F.scaled_dot_product_attention(
                        q_4d, k_4d, v_4d,
                        attn_mask=None,
                        dropout_p=self.dropout.p if self.training else 0.0,
                        is_causal=True
                    )

                # 转换回3维和原始数据类型
                attn_output = attn_output.transpose(1, 2).contiguous().view(B, T, self.head_embd)
                attn_output = attn_output.to(input_x.dtype)
                return attn_output
            except RuntimeError as e:
                print(f"Flash Attention failed, falling back to manual implementation: {e}")

        # 普通实现（确保数据类型一致）
        q, k, v = q.to(input_x.dtype), k.to(input_x.dtype), v.to(input_x.dtype)
        wei = q @ k.transpose(-2, -1) * (self.head_embd ** -0.5)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        attn_output = wei @ v

        return attn_output

In [8]:
class BingramLanguageModel(nn.Module):
    def __init__(self, block_size, vocab_size, n_embd, n_head, n_layer, dropout):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head, dropout=dropout) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)
        self.block_size = block_size  # 添加block_size属性

    def forward(self, idx, targets=None):
        B, T = idx.shape
        tok_emb = self.token_embedding_table(idx)  # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=idx.device)) # (T,C)
        x = tok_emb + pos_emb  # (B, T, C)
        x = self.blocks(x)  # (B, T, C)
        x = self.ln_f(x)
        logits = self.lm_head(x)  # (B,T, vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.reshape(B * T, C)
            targets = targets.reshape(B * T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    # 添加generate方法用于文本生成
    def generate(self, idx, max_new_tokens):
        # idx是当前上下文索引数组，形状为(B, T)
        for _ in range(max_new_tokens):
            # 裁剪idx到block_size长度
            idx_cond = idx[:, -self.block_size:]
            # 获取预测
            logits, _ = self(idx_cond)
            # 聚焦最后时间步
            logits = logits[:, -1, :]  # 变为(B, C)
            # 应用softmax获取概率
            probs = F.softmax(logits, dim=-1)  # (B, C)
            # 从分布中采样
            idx_next = torch.multinomial(probs, num_samples=1)  # (B, 1)
            # 将采样索引附加到序列中
            idx = torch.cat((idx, idx_next), dim=1)  # (B, T+1)
        return idx

In [9]:
# 模型训练数据集
block_size = 8
batch_size = 256
max_iter = 500
learn_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
n_embd = 512
eval_interval = 100
eval_iters = 100
head_size = 64
num_layers = 8
dropout = 0.1

In [10]:
# 读取文本文件
with open('/kaggle/input/san-guo-txt/san_guo.txt') as f:
    text = f.read()

# 字典、编码器(函数)、解码器(函数)
chars = sorted(list(set(text)))
vocab_size = len(chars)
stoi = {ch:i for i,ch in enumerate(chars)}  #str_to_index
itos = {i:ch for i,ch in enumerate(chars)}  #index_to_str

encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

# 文本转换token index
data = torch.tensor(encode(text), dtype=torch.long)

# 拆分数据集
n = int(len(data) * .9)
train_data = data[:n]
val_data = data[n:]

# 模型训练
model = BingramLanguageModel(block_size, vocab_size, n_embd, head_size, num_layers, dropout)
model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=learn_rate)

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

# 使用tqdm创建进度条
progress_bar = tqdm(range(max_iter), desc="Training Progress")

for iter in progress_bar:

    if iter % eval_interval == 0 or iter == max_iter - 1:
        losses = estimate_loss()
        progress_bar.set_postfix({
            # 'train_loss': f"{losses['train']:.4f}",
            # 'val_loss': f"{losses['val']:.4f}"
        })
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}\n")

    # 批次样本
    xb, yb = get_batch('train')

    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# 模型生成
idx = torch.zeros((1,1), dtype=torch.long, device=device)
print("\nGenerated Text:")
print(decode(model.generate(idx, max_new_tokens=500)[0].tolist()))

Training Progress:   0%|          | 0/500 [00:00<?, ?it/s]/usr/lib/python3.11/contextlib.py:105: FutureWarning: `torch.backends.cuda.sdp_kernel()` is deprecated. In the future, this context manager will be removed. Please see `torch.nn.attention.sdpa_kernel()` for the new context manager, with updated signature.
  self.gen = func(*args, **kwds)
Training Progress:   0%|          | 0/500 [00:31<?, ?it/s]

step 0: train loss 8.4885, val loss 8.4845



Training Progress:  20%|██        | 100/500 [01:53<03:11,  2.09it/s]

step 100: train loss 5.2095, val loss 5.5107



Training Progress:  40%|████      | 200/500 [03:19<02:41,  1.86it/s]  

...

step 400: train loss 4.1433, val loss 4.8050



Training Progress: 100%|█████████▉| 499/500 [07:38<00:00,  1.92it/s]

step 499: train loss 3.9998, val loss 4.7831



Training Progress: 100%|██████████| 500/500 [07:38<00:00,  1.09it/s]



Generated Text:

　　且说张郃自出兵有二十里，大夫去见马超。朱灵接着，方告文事。韩福乃喜曰：“汝乃荀彧奏，远征伐纣面少，临行。孔明见帐后亲自乱，忽见汉中风抖擞资，威风抖擞，鼓角泉，实测边砖。截秦朗弼于领军山子。百官皆认得惊，乃别“大臣臣夫人也！”蒙曰：“天长只见小儿光烛！”众乃问曰：“可由将主公将，岂不见陛下功名玺！”拍马执归献人来降；却为副将，乃送待与那。”。忽报曰：“孔明在鄱阳原中，令人忙应夏侯楙随苦谏曰：“筵师双部将何期又见众人穷凶来，对薄界，停说曹军事。‘臣领了王位于汉中，随后出兵。程侧报知其事，寻贤日间，忽听炮墙大进；白马军法，地离白家，事不一将，感德忠德轮戟，谓左右囷葬之。”遂深沟高，一夜取以安天下也！”域闻奏，事在马上，乃追吕布平、文聘、砖二将，尊地八魂。”夏侯楙让械访旨。因云长曰：“吾故走耶小沛，兵不反失语。兴、吴懿早复彼矣！”孔明叹曰：“教军在何处？”懿曰：“蜀名将召，存区区，望后面三队，遮兵去了。彼百姓，侵水面牙大患，先早晚投降书与逊探薄略，具行官渡江之上。”二人闻之，曰：“连夜来新时去桃小童襟；若不来接雁，必反写来！”玄伐个大鏖兵四散顿丘民，急令丁原。离射绿车驾，权问曰：“贤焉腑
